In [1]:
import numpy as np
import cma
import cma.purecma as purecma
import matplotlib.pyplot as plt

# Formes Canoniques d'Algorithmes Evolutionnaires

## Nomenclature générale d'un Algorithme Evolutionnaire : 
Un algorithme evolutionnaire reprend géneralement les composantes suivantes : 
* Une population d'individus (un ou plusieurs) representant des génotypes 
* Une méthode de perturbation des individus pour générer des nouveaux individus (successeurs ou offspring)
* Une fonction d'évaluation qui sert a comparer les genotypes en fonction de leur phenotypes et a en sélectionner certains.

Donc pour chaque élément les variantes les plus communes sont : 
- Pour la population : 1 Si elle se compose d'un seul élement $\mu$ si plusieurs et $\mu / \sigma$ si parmi la population un nombre $\sigma$ seulement participent a la créations des successeurs.
- Successeurs : 1 ou $\lambda$ si plusieurs successeurs.
- Mode de sélection "," si la sélection se fait parmi l'offspring ou "+" si elle se fait parmi les parents et l'offspring.

Donc par exemple : 
- (1,1) ne peut pas exister ça voudrait dire qu'on sélectionne parmi une offspring de 1 donc enfaite le nouvel élément serrait seul et on n'aurait pas avec qui le comparer on le prendrait systématiquement
- (1+1) signifie qu'on a genere un individu, qu'on perturbe a chaque étape et qu'on garde l'offspring si elle est meilleur que l'individu (donc on effectue une sélection sur le parent et l'offspring).
- (1,$\lambda$) veut dire que a chaque étape on génère plusieures descendants et on remplace le parent systématiquement par le meilleur élément de l'offspring (le parent ne participe pas a la sélection).
- ($\mu$ + $\lambda$) veut dire que la sélection de la prochaine population qui contient plusieurs individus se fait parmi plusieurs l'offspring et l'ancienne population. 

## Algorithmes evolutionnaires basiques
Dans cette partie nous allons implémenter et comparer différents algorithmes génétiques simples en les testant sur le problème du Lab 01 sur la prédiction des prix des maisons dans boston.

In [2]:
"""
On commence par l'algorithme le plus simple qu'on puisse imaginer, un (1+1) avec perturbation normale.
"""
def launch_oneplusone(center, sigma, ma_func, nbeval=10000):
    parent = np.array(center)
    i=0
    j=0
    parentFit = ma_func(parent)
    bestFit = parentFit
    bestIt = 0
    while i<nbeval:
        child = parent + sigma*np.random.normal(0,np.ones(parent.shape),parent.shape)
        childFit = ma_func(child)
        if childFit <= parentFit:
            parentFit = childFit
            parent = np.copy(child)
            if bestFit > parentFit:
                bestFit = parentFit
                bestIt = i
                print("New best fit found : ", bestFit," a l'itteration : ",i)

        j+=1
        solutions = np.array([parent])
        i+=1
    print ("Best fit",bestFit,"at iteration",bestIt)
    return bestFit


In [3]:
"""
Dans ce qui suit nous utiliserons la structure de réseaux de neurones simple que nous avons vu précédement
car elle permet de manipuler les poids de façon "flat" c'est a dire en amount (de façon non structurée)
(Pas la peine de relire tout ça c'est pas important, c'est a titre indicatif et on aurait pu utiliser scikit-learn
remarquez juste que j'ai mis une fonction d'activation linéaire a la place de la sigmoid)
"""
def sigmoid(x):
    return 1./(1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

class SimpleNeuralControllerNumpy():
    def __init__(self, n_in, n_out, n_hidden_layers=2, n_neurons_per_hidden=5, params=None):
        self.dim_in = n_in
        self.dim_out = n_out
        # if params is provided, we look for the number of hidden layers and neuron per layer into that parameter (a dicttionary)
        if (not params==None):
            if ("n_hidden_layers" in params.keys()):
                n_hidden_layers=params["n_hidden_layers"]
            if ("n_neurons_per_hidden" in params.keys()):
                n_neurons_per_hidden=params["n_neurons_per_hidden"]
        self.n_per_hidden = n_neurons_per_hidden
        self.n_hidden_layers = n_hidden_layers
        self.weights = None 
        self.n_weights = None
        self.init_random_params()
        self.out = np.zeros(n_out)
        #print("Creating a simple mlp with %d inputs, %d outputs, %d hidden layers and %d neurons per layer"%(n_in, n_out,n_hidden_layers, n_neurons_per_hidden))
    def init_random_params(self):
        if(self.n_hidden_layers > 0):
            self.weights = [np.random.random((self.dim_in,self.n_per_hidden))] # In -> first hidden
            self.bias = [np.random.random(self.n_per_hidden)] # In -> first hidden
            for i in range(self.n_hidden_layers-1): # Hidden -> hidden
                self.weights.append(np.random.random((self.n_per_hidden,self.n_per_hidden)))
                self.bias.append(np.random.random(self.n_per_hidden))
            self.weights.append(np.random.random((self.n_per_hidden,self.dim_out))) # -> last hidden -> out
            self.bias.append(np.random.random(self.dim_out))
        else:
            self.weights = [np.random.random((self.dim_in,self.dim_out))] # Single-layer perceptron
            self.bias = [np.random.random(self.dim_out)]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])

    def get_parameters(self):
        """
        Returns all network parameters as a single array
        """
        flat_weights = np.hstack([arr.flatten() for arr in (self.weights+self.bias)])
        return flat_weights

    def set_parameters(self, flat_parameters):
        """
        Set all network parameters from a single array
        """
        i = 0 # index
        to_set = []
        self.weights = list()
        self.bias = list()
        if(self.n_hidden_layers > 0):
            # In -> first hidden
            w0 = np.array(flat_parameters[i:(i+self.dim_in*self.n_per_hidden)])
            self.weights.append(w0.reshape(self.dim_in,self.n_per_hidden))
            i += self.dim_in*self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                w = np.array(flat_parameters[i:(i+self.n_per_hidden*self.n_per_hidden)])
                self.weights.append(w.reshape((self.n_per_hidden,self.n_per_hidden)))
                i += self.n_per_hidden*self.n_per_hidden
            # -> last hidden -> out
            wN = np.array(flat_parameters[i:(i+self.n_per_hidden*self.dim_out)])
            self.weights.append(wN.reshape((self.n_per_hidden,self.dim_out)))
            i += self.n_per_hidden*self.dim_out
            # Samefor bias now
            # In -> first hidden
            b0 = np.array(flat_parameters[i:(i+self.n_per_hidden)])
            self.bias.append(b0)
            i += self.n_per_hidden
            for l in range(self.n_hidden_layers-1): # Hidden -> hidden
                b = np.array(flat_parameters[i:(i+self.n_per_hidden)])
                self.bias.append(b)
                i += self.n_per_hidden
            # -> last hidden -> out
            bN = np.array(flat_parameters[i:(i+self.dim_out)])
            self.bias.append(bN)
            i += self.dim_out
        else:
            n_w = self.dim_in*self.dim_out
            w = np.array(flat_parameters[:n_w])
            self.weights = [w.reshape((self.dim_in,self.dim_out))]
            self.bias = [np.array(flat_parameters[n_w:])]
        self.n_weights = np.sum([np.product(w.shape) for w in self.weights]) + np.sum([np.product(b.shape) for b in self.bias])
    
    def predict(self,x):
        if(self.n_hidden_layers > 0):
            #Input
            a = np.matmul(x,self.weights[0]) + self.bias[0]
            #y = sigmoid(a)
            y = a
            # hidden -> hidden
            for i in range(1,self.n_hidden_layers-1):
                a = np.matmul(y, self.weights[i]) + self.bias[i]
                y = sigmoid(a)
            # Out
            a = np.matmul(y, self.weights[-1]) + self.bias[-1]
            """
            ICI !! 
            """
            #out = tanh(a)
            out = a
            return out
        else: # Simple monolayer perceptron
            """
            et ICI !! 
            """
            #return tanh(np.matmul(x,self.weights[0]) + self.bias[0])
            return np.matmul(x,self.weights[0]) + self.bias[0]

In [4]:
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
#On charge les données 
#On utilise un réseau de neurones simple 
Network = SimpleNeuralControllerNumpy(13, 1, n_hidden_layers=0, n_neurons_per_hidden=0, params=None)
Network.init_random_params()
print(Network.get_parameters())

[0.57687309 0.44849256 0.90440604 0.27935665 0.64800209 0.27783575
 0.75146805 0.93848309 0.58925682 0.61058508 0.34585676 0.44325241
 0.94160259 0.88539774]


In [5]:
#La fonction d'evalutation qu'on prend va associer a chaque vecteur de poids w l'erreur moyenne quadratique

def evaluation_function(w):
    X, y = load_boston(return_X_y=True)
    Network = SimpleNeuralControllerNumpy(13, 1, n_hidden_layers=0, n_neurons_per_hidden=0, params=None)
    Network.set_parameters(w)
    y_pred = Network.predict(X)
    return mean_squared_error(y,y_pred)

Network = SimpleNeuralControllerNumpy(13, 1, n_hidden_layers=0, n_neurons_per_hidden=0, params=None)
Network.init_random_params()
init_point = Network.get_parameters()
evaluation_function(init_point)


44442.24302718403

In [6]:
launch_oneplusone(init_point, 0.01, evaluation_function, nbeval=10000)

New best fit found :  42809.19030652453  a l'itteration :  0
New best fit found :  40970.191734336375  a l'itteration :  2
New best fit found :  37291.91802314245  a l'itteration :  3
New best fit found :  34909.35195893095  a l'itteration :  5
New best fit found :  34239.937087147955  a l'itteration :  9
New best fit found :  32899.42039318274  a l'itteration :  10
New best fit found :  31425.335743390424  a l'itteration :  11
New best fit found :  30962.61191612771  a l'itteration :  12
New best fit found :  27808.123324196196  a l'itteration :  13
New best fit found :  26386.22107701765  a l'itteration :  15
New best fit found :  23760.76264959841  a l'itteration :  16
New best fit found :  22725.329685467023  a l'itteration :  18
New best fit found :  22558.14947368113  a l'itteration :  20
New best fit found :  21366.02824332318  a l'itteration :  23
New best fit found :  19907.032450161918  a l'itteration :  24
New best fit found :  17051.885762934282  a l'itteration :  26
New be

New best fit found :  88.47735315382266  a l'itteration :  1840
New best fit found :  88.44273021536206  a l'itteration :  1849
New best fit found :  88.40130178961768  a l'itteration :  1895
New best fit found :  87.96020083606192  a l'itteration :  1917
New best fit found :  86.86938930661796  a l'itteration :  1995
New best fit found :  86.68901010381434  a l'itteration :  2094
New best fit found :  86.42158382495123  a l'itteration :  2100
New best fit found :  85.72307999177957  a l'itteration :  2173
New best fit found :  85.69300386865645  a l'itteration :  2189
New best fit found :  83.61104032589128  a l'itteration :  2292
New best fit found :  83.17524588788646  a l'itteration :  2359
New best fit found :  82.97359812648865  a l'itteration :  2369
New best fit found :  82.87509626657344  a l'itteration :  2394
New best fit found :  82.44417313232944  a l'itteration :  2403
New best fit found :  81.85567141331988  a l'itteration :  2451
New best fit found :  80.97987573870077 

KeyboardInterrupt: 

In [7]:
"""
Petit rafinement de l'algorithme en ajoutant la règle des 1/5.
Remarque : Ce rafinement contribue a réduire la stochacité de l'algorithme et donc sont utilisation doit être
bien mesurée car elle le rend trés dépendant du point de départ
"""

def launch_ESonefifthRule(center, sigma, ma_func, nbeval=10000):
    parent = np.array(center)
    i = 0
    j = 0
    parentFit = ma_func(parent)
    bestFit = parentFit
    bestIt = 0
    print("Parent : ", parent)
    while i < nbeval:
        child = parent + sigma * np.random.normal(0, np.ones(parent.shape), parent.shape)
        childFit = ma_func(child)
        if childFit <= parentFit:
            parentFit = childFit
            parent = np.copy(child)
            sigma = sigma*2
            if bestFit > parentFit:
                bestFit = parentFit
                bestIt = i
                print("New best fit found : ", bestFit," a l'itteration : ",i)

        else:
            sigma = sigma*(2**(-(1/4)))
        j += 1
        solutions = np.array([parent])
        i += 1
    print("Best fit", bestFit, "at iteration", bestIt)
    return bestFit


launch_ESonefifthRule(init_point, 0.1, evaluation_function, nbeval=10000)

Parent :  [0.0698919  0.20766986 0.53785086 0.81710399 0.13612967 0.96433433
 0.41511499 0.15137555 0.01988385 0.07470285 0.68444435 0.37053039
 0.81576308 0.80510281]
New best fit found :  44408.282492179234  a l'itteration :  22
New best fit found :  44124.26665745583  a l'itteration :  23
New best fit found :  43461.86072964179  a l'itteration :  27
New best fit found :  38937.12347875447  a l'itteration :  29
New best fit found :  38485.76550663952  a l'itteration :  30
New best fit found :  34628.2737711754  a l'itteration :  31
New best fit found :  30353.323211684103  a l'itteration :  32
New best fit found :  12079.146885298298  a l'itteration :  36
New best fit found :  6459.3078993019  a l'itteration :  37
New best fit found :  914.1966822830221  a l'itteration :  39
New best fit found :  471.51953520685646  a l'itteration :  47
New best fit found :  456.27861392123174  a l'itteration :  58
New best fit found :  444.0126307506866  a l'itteration :  68
New best fit found :  43

New best fit found :  198.00329761818205  a l'itteration :  684
New best fit found :  197.86883427519925  a l'itteration :  691
New best fit found :  197.37026372716096  a l'itteration :  694
New best fit found :  197.2521559904228  a l'itteration :  703
New best fit found :  197.14363608242022  a l'itteration :  714
New best fit found :  197.05994593249613  a l'itteration :  715
New best fit found :  196.84454976285437  a l'itteration :  719
New best fit found :  196.36665227237827  a l'itteration :  720
New best fit found :  196.1111483733567  a l'itteration :  726
New best fit found :  196.0064272636108  a l'itteration :  737
New best fit found :  195.96453288463997  a l'itteration :  741
New best fit found :  195.82632647589645  a l'itteration :  744
New best fit found :  195.7102561759987  a l'itteration :  750
New best fit found :  195.45749783856306  a l'itteration :  753
New best fit found :  195.26280439041068  a l'itteration :  757
New best fit found :  195.2354376808284  a l

New best fit found :  163.36638216823195  a l'itteration :  1365
New best fit found :  163.3285178336529  a l'itteration :  1374
New best fit found :  162.51716775351184  a l'itteration :  1376
New best fit found :  162.48627477342419  a l'itteration :  1383
New best fit found :  161.99711146909326  a l'itteration :  1384
New best fit found :  161.87667815516173  a l'itteration :  1387
New best fit found :  161.50171795288227  a l'itteration :  1393
New best fit found :  161.49622330168225  a l'itteration :  1402
New best fit found :  161.38150181361894  a l'itteration :  1408
New best fit found :  161.3565574237027  a l'itteration :  1411
New best fit found :  161.3084687973751  a l'itteration :  1421
New best fit found :  161.2616999795073  a l'itteration :  1423
New best fit found :  161.25809189217554  a l'itteration :  1426
New best fit found :  160.96985349427018  a l'itteration :  1433
New best fit found :  160.95001719384112  a l'itteration :  1442
New best fit found :  160.928

New best fit found :  140.38158674536584  a l'itteration :  2011
New best fit found :  140.36198922344175  a l'itteration :  2012
New best fit found :  140.29467511077104  a l'itteration :  2017
New best fit found :  140.195048160826  a l'itteration :  2018
New best fit found :  140.10511847987078  a l'itteration :  2025
New best fit found :  140.07548232511206  a l'itteration :  2032
New best fit found :  139.95038987706582  a l'itteration :  2033
New best fit found :  139.88501601537462  a l'itteration :  2041
New best fit found :  139.81733988012064  a l'itteration :  2046
New best fit found :  139.81138652228486  a l'itteration :  2049
New best fit found :  139.76142386180652  a l'itteration :  2051
New best fit found :  139.3378968915299  a l'itteration :  2053
New best fit found :  139.01596086069236  a l'itteration :  2061
New best fit found :  139.0076253018258  a l'itteration :  2063
New best fit found :  138.72692081209357  a l'itteration :  2064
New best fit found :  138.004

New best fit found :  120.16436932691359  a l'itteration :  2686
New best fit found :  120.03752341608444  a l'itteration :  2690
New best fit found :  119.95533453004916  a l'itteration :  2701
New best fit found :  119.73876592932811  a l'itteration :  2702
New best fit found :  119.59078322588437  a l'itteration :  2706
New best fit found :  119.46963991700409  a l'itteration :  2712
New best fit found :  119.44981091034735  a l'itteration :  2728
New best fit found :  119.40899080227878  a l'itteration :  2730
New best fit found :  119.35540953849126  a l'itteration :  2732
New best fit found :  119.24070342210109  a l'itteration :  2734
New best fit found :  119.20159161982085  a l'itteration :  2743
New best fit found :  119.19369608010288  a l'itteration :  2750
New best fit found :  119.19225548002885  a l'itteration :  2753
New best fit found :  119.17435819128208  a l'itteration :  2757
New best fit found :  119.14354996303834  a l'itteration :  2759
New best fit found :  119

...

New best fit found :  71.9150343929661  a l'itteration :  6682
New best fit found :  71.90196589024653  a l'itteration :  6685
New best fit found :  71.89848900524208  a l'itteration :  6696
New best fit found :  71.89646069496932  a l'itteration :  6698
New best fit found :  71.88379268051358  a l'itteration :  6700
New best fit found :  71.86425310417991  a l'itteration :  6706
New best fit found :  71.86415410913027  a l'itteration :  6707
New best fit found :  71.81442778412628  a l'itteration :  6709
New best fit found :  71.75773555351125  a l'itteration :  6713
New best fit found :  71.75492531166842  a l'itteration :  6717
New best fit found :  71.74665353062079  a l'itteration :  6718
New best fit found :  71.74506186098293  a l'itteration :  6730
New best fit found :  71.69585212335268  a l'itteration :  6733
New best fit found :  71.6903999893286  a l'itteration :  6740
New best fit found :  71.6627148632966  a l'itteration :  6747
New best fit found :  71.6566901029323  a l

New best fit found :  68.15337565431379  a l'itteration :  7356
New best fit found :  68.1380676125633  a l'itteration :  7359
New best fit found :  68.12865952937582  a l'itteration :  7361
New best fit found :  68.12117677269084  a l'itteration :  7373
New best fit found :  68.11875890943723  a l'itteration :  7375
New best fit found :  68.08989075827836  a l'itteration :  7376
New best fit found :  68.06574843733318  a l'itteration :  7384
New best fit found :  68.0580385989217  a l'itteration :  7389
New best fit found :  68.05776033214272  a l'itteration :  7390
New best fit found :  68.02951555217271  a l'itteration :  7396
New best fit found :  67.90337111000764  a l'itteration :  7397
New best fit found :  67.87482706679023  a l'itteration :  7404
New best fit found :  67.83023952704612  a l'itteration :  7405
New best fit found :  67.69448234897853  a l'itteration :  7411
New best fit found :  67.66976894114575  a l'itteration :  7422
New best fit found :  67.66441429873043  a

New best fit found :  64.55781351822174  a l'itteration :  8033
New best fit found :  64.50991959947828  a l'itteration :  8040
New best fit found :  64.50390515261883  a l'itteration :  8048
New best fit found :  64.48648624856001  a l'itteration :  8055
New best fit found :  64.46849536341634  a l'itteration :  8062
New best fit found :  64.46038965666528  a l'itteration :  8067
New best fit found :  64.44649476953951  a l'itteration :  8069
New best fit found :  64.43707031072597  a l'itteration :  8071
New best fit found :  64.43120491460905  a l'itteration :  8080
New best fit found :  64.41777525064109  a l'itteration :  8089
New best fit found :  64.40968515869295  a l'itteration :  8093
New best fit found :  64.3643176529016  a l'itteration :  8094
New best fit found :  64.34435455094291  a l'itteration :  8095
New best fit found :  64.24490916643627  a l'itteration :  8096
New best fit found :  64.13787695941167  a l'itteration :  8103
New best fit found :  64.12688852312648  

New best fit found :  61.703773834133656  a l'itteration :  8695
New best fit found :  61.697807388325614  a l'itteration :  8699
New best fit found :  61.68851825940673  a l'itteration :  8708
New best fit found :  61.67869844860203  a l'itteration :  8716
New best fit found :  61.66333337572742  a l'itteration :  8717
New best fit found :  61.658563191050746  a l'itteration :  8721
New best fit found :  61.648798938335844  a l'itteration :  8729
New best fit found :  61.64829465488555  a l'itteration :  8732
New best fit found :  61.635308069135206  a l'itteration :  8735
New best fit found :  61.62168824553452  a l'itteration :  8737
New best fit found :  61.59907311298592  a l'itteration :  8741
New best fit found :  61.59666567099994  a l'itteration :  8751
New best fit found :  61.58907921153627  a l'itteration :  8757
New best fit found :  61.57402029668071  a l'itteration :  8760
New best fit found :  61.57018419658121  a l'itteration :  8763
New best fit found :  61.5003895195

KeyboardInterrupt: 

In [8]:
def launch_LambdaES(center, _lambda, sigma,ma_func, nbeval=10000):
    center = np.array(center)
    parent = center + sigma * np.random.normal(0, center.shape, center.shape[0])
    i = 0
    j = 0
    parentFit = ma_func(parent)
    bestFit = parentFit
    bestIt = 0
    print("Parent : ", parent)
    while i < nbeval:
        for k in range(_lambda):
            child = parent + sigma * np.random.normal(0, np.ones(parent.shape), parent.shape)
            childFit = ma_func(child)
            if childFit <= parentFit:
                parentFit = childFit
                parent = np.copy(child)
                sigma = sigma * 2
                if bestFit > parentFit:
                    bestFit = parentFit
                    bestIt = i
                    print("New best fit found : ", bestFit," a l'itteration : ",i)
            else:
                sigma = sigma * (2 ** (-(1 / 4)))
            j += 1
            solutions = np.array([parent])
        i += 1
    print("Best fit", bestFit, "at iteration", bestIt)
    return bestFit

launch_LambdaES(init_point,100, 0.01, evaluation_function, nbeval=1000)

Parent :  [ 0.01855652  0.53732642  0.59293353  0.86341082  0.02191658  1.13623698
  0.46051785  0.28086657  0.01449371 -0.14701793  0.7613408   0.57608863
  0.57677654  0.88189818]
New best fit found :  42461.98425054206  a l'itteration :  0
New best fit found :  39562.15661971355  a l'itteration :  0
New best fit found :  37601.93023414073  a l'itteration :  0
New best fit found :  34069.08148252634  a l'itteration :  0
New best fit found :  30945.841649945352  a l'itteration :  0
New best fit found :  28431.55457598584  a l'itteration :  0
New best fit found :  14433.104238537355  a l'itteration :  0
New best fit found :  6064.2817921868245  a l'itteration :  0
New best fit found :  4626.735556244977  a l'itteration :  0
New best fit found :  3031.5298903750145  a l'itteration :  0
New best fit found :  1858.647667801797  a l'itteration :  0
New best fit found :  1400.6900855515723  a l'itteration :  0
New best fit found :  1384.096113960797  a l'itteration :  0
New best fit found :

New best fit found :  178.6504561161243  a l'itteration :  7
New best fit found :  176.616734583651  a l'itteration :  7
New best fit found :  176.02420745506635  a l'itteration :  7
New best fit found :  175.041426408354  a l'itteration :  7
New best fit found :  174.67151000340954  a l'itteration :  7
New best fit found :  173.35802566183878  a l'itteration :  7
New best fit found :  172.6381482206012  a l'itteration :  7
New best fit found :  172.57191890967843  a l'itteration :  7
New best fit found :  172.0606919697681  a l'itteration :  7
New best fit found :  170.87549944528118  a l'itteration :  7
New best fit found :  170.42159424529376  a l'itteration :  7
New best fit found :  170.17192368477026  a l'itteration :  7
New best fit found :  166.988163482671  a l'itteration :  7
New best fit found :  166.08274270201028  a l'itteration :  7
New best fit found :  165.98102286599246  a l'itteration :  7
New best fit found :  162.26252393783676  a l'itteration :  7
New best fit foun

New best fit found :  64.39927406921497  a l'itteration :  14
New best fit found :  60.569878540700444  a l'itteration :  14
New best fit found :  60.094361800798254  a l'itteration :  14
New best fit found :  60.052939430857236  a l'itteration :  14
New best fit found :  60.03317858956157  a l'itteration :  14
New best fit found :  60.026708514589046  a l'itteration :  14
New best fit found :  60.02529846060476  a l'itteration :  14
New best fit found :  60.010573970996944  a l'itteration :  14
New best fit found :  59.86717617729275  a l'itteration :  14
New best fit found :  59.82995669986396  a l'itteration :  14
New best fit found :  59.827974596093824  a l'itteration :  14
New best fit found :  59.72942435366541  a l'itteration :  14
New best fit found :  59.704934139780796  a l'itteration :  14
New best fit found :  59.6560373841193  a l'itteration :  14
New best fit found :  59.56458795318207  a l'itteration :  14
New best fit found :  59.551468311749794  a l'itteration :  14
N

New best fit found :  50.40504594561447  a l'itteration :  21
New best fit found :  50.3500186107005  a l'itteration :  21
New best fit found :  50.30054065916358  a l'itteration :  21
New best fit found :  50.26023611579706  a l'itteration :  21
New best fit found :  50.25162897964923  a l'itteration :  21
New best fit found :  50.216872362798846  a l'itteration :  21
New best fit found :  50.15759680632292  a l'itteration :  21
New best fit found :  50.12401769991954  a l'itteration :  21
New best fit found :  50.10519773854949  a l'itteration :  21
New best fit found :  50.08065225279873  a l'itteration :  21
New best fit found :  50.07379439776447  a l'itteration :  21
New best fit found :  50.059034792219  a l'itteration :  21
New best fit found :  50.04265687295132  a l'itteration :  21
New best fit found :  49.971145496455236  a l'itteration :  21
New best fit found :  49.96446281159251  a l'itteration :  21
New best fit found :  49.95793639628853  a l'itteration :  21
New best 

New best fit found :  46.83073275699498  a l'itteration :  27
New best fit found :  46.8245476912085  a l'itteration :  28
New best fit found :  46.79819184006627  a l'itteration :  28
New best fit found :  46.78018998037726  a l'itteration :  28
New best fit found :  46.77675008348809  a l'itteration :  28
New best fit found :  46.76848434148575  a l'itteration :  28
New best fit found :  46.7591120407305  a l'itteration :  28
New best fit found :  46.758056746249984  a l'itteration :  28
New best fit found :  46.75553011912994  a l'itteration :  28
New best fit found :  46.753996600658205  a l'itteration :  28
New best fit found :  46.75053530042634  a l'itteration :  28
New best fit found :  46.748279495012184  a l'itteration :  28
New best fit found :  46.739150786637815  a l'itteration :  28
New best fit found :  46.73576025033236  a l'itteration :  28
New best fit found :  46.73370056507879  a l'itteration :  28
New best fit found :  46.73144481207562  a l'itteration :  28
New be

...

New best fit found :  45.340076031889005  a l'itteration :  54
New best fit found :  45.339682986384325  a l'itteration :  54
New best fit found :  45.33957704732801  a l'itteration :  55
New best fit found :  45.339467752344426  a l'itteration :  55
New best fit found :  45.33944588217787  a l'itteration :  55
New best fit found :  45.339081263210694  a l'itteration :  55
New best fit found :  45.33900528714089  a l'itteration :  55
New best fit found :  45.3383294660913  a l'itteration :  55
New best fit found :  45.338097463560565  a l'itteration :  55
New best fit found :  45.338001169332216  a l'itteration :  55
New best fit found :  45.337849707528726  a l'itteration :  55
New best fit found :  45.33783707823592  a l'itteration :  55
New best fit found :  45.33732166412131  a l'itteration :  55
New best fit found :  45.336458776692496  a l'itteration :  55
New best fit found :  45.33612620327662  a l'itteration :  55
New best fit found :  45.336122319645135  a l'itteration :  55


New best fit found :  45.20122935924732  a l'itteration :  61
New best fit found :  45.198962632371675  a l'itteration :  61
New best fit found :  45.198776793796526  a l'itteration :  61
New best fit found :  45.19841429079679  a l'itteration :  61
New best fit found :  45.19471881577135  a l'itteration :  61
New best fit found :  45.19438069251926  a l'itteration :  61
New best fit found :  45.19436002616995  a l'itteration :  61
New best fit found :  45.19271802513762  a l'itteration :  61
New best fit found :  45.19017294296016  a l'itteration :  62
New best fit found :  45.18849115658992  a l'itteration :  62
New best fit found :  45.186686431172944  a l'itteration :  62
New best fit found :  45.18326221290505  a l'itteration :  62
New best fit found :  45.17963018467613  a l'itteration :  62
New best fit found :  45.17830133329028  a l'itteration :  62
New best fit found :  45.17819817126013  a l'itteration :  62
New best fit found :  45.17802874971194  a l'itteration :  62
New b

New best fit found :  45.09467318931231  a l'itteration :  68
New best fit found :  45.09429434794092  a l'itteration :  68
New best fit found :  45.094113379808746  a l'itteration :  68
New best fit found :  45.09364626600673  a l'itteration :  68
New best fit found :  45.09269947025746  a l'itteration :  68
New best fit found :  45.09252496851428  a l'itteration :  68
New best fit found :  45.09244428513158  a l'itteration :  68
New best fit found :  45.09126487675367  a l'itteration :  68
New best fit found :  45.09096626966076  a l'itteration :  68
New best fit found :  45.08832507459791  a l'itteration :  68
New best fit found :  45.088175247550566  a l'itteration :  69
New best fit found :  45.08782081095747  a l'itteration :  69
New best fit found :  45.08731668418318  a l'itteration :  69
New best fit found :  45.08556929268237  a l'itteration :  69
New best fit found :  45.08422201487395  a l'itteration :  69
New best fit found :  45.083890697350405  a l'itteration :  69
New b

New best fit found :  44.993707832492866  a l'itteration :  75
New best fit found :  44.99364395819004  a l'itteration :  75
New best fit found :  44.993449679397486  a l'itteration :  75
New best fit found :  44.99340846393488  a l'itteration :  75
New best fit found :  44.99323751749736  a l'itteration :  75
New best fit found :  44.993145517200944  a l'itteration :  75
New best fit found :  44.993040125032216  a l'itteration :  75
New best fit found :  44.992965590754466  a l'itteration :  75
New best fit found :  44.99277761390637  a l'itteration :  75
New best fit found :  44.99266532036011  a l'itteration :  75
New best fit found :  44.99239323896116  a l'itteration :  75
New best fit found :  44.99190010473546  a l'itteration :  75
New best fit found :  44.991834421740904  a l'itteration :  75
New best fit found :  44.9916348730689  a l'itteration :  76
New best fit found :  44.99151699915106  a l'itteration :  76
New best fit found :  44.991488021472115  a l'itteration :  76
Ne

KeyboardInterrupt: 

In [9]:
res = cma.CMAEvolutionStrategy(init_point, 0.01).optimize(evaluation_function,maxfun=10000).result


(5_w,11)-aCMA-ES (mu_w=3.4,w_1=42%) in dimension 14 (seed=428287, Tue Apr  7 18:47:35 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     11 4.137903966916115e+04 1.0e+00 9.22e-03  9e-03  9e-03 0:00.0
    2     22 3.971498130752716e+04 1.1e+00 9.85e-03  9e-03  1e-02 0:00.1
    3     33 3.705961133689649e+04 1.2e+00 1.02e-02  1e-02  1e-02 0:00.1
   78    858 1.408101186205847e+02 6.9e+00 5.05e-02  2e-02  6e-02 0:03.2
  100   1100 7.820373997838625e+01 1.5e+01 9.50e-02  3e-02  1e-01 0:04.0
  200   2200 3.190825866115980e+01 9.9e+01 4.38e-02  4e-03  8e-02 0:07.9
  300   3300 2.465607948778208e+01 3.2e+02 3.53e-02  2e-03  9e-02 0:11.7
  400   4400 2.398156281947540e+01 1.4e+03 2.95e-02  7e-04  1e-01 0:15.6
  500   5500 2.359103528545178e+01 3.0e+03 7.01e-02  9e-04  3e-01 0:19.3
  600   6600 2.193852644011352e+01 1.1e+04 4.58e-02  4e-04  5e-01 0:24.1
  700   7700 2.189483754264236e+01 1.4e+04 1.71e-03  9e-06  1e-02 0:28.2
  800   8800 2.189483118174181e+0

In [10]:
res = purecma.fmin(evaluation_function,init_point,0.01,maxfevals=10000)

evals: ax-ratio max(std)   f-value
   11:     1.0  9.8e-03  42256.52069719685
   22:     1.0  9.8e-03  39343.19932329578
 1100:     2.5  2.9e-02  167.44382899993926
 2200:    10.3  1.1e-01  69.97955269169346
 3300:    27.4  4.4e-02  35.9525148801903
 4400:    60.9  5.8e-02  31.768337501730766
 5500:   264.1  7.7e-02  24.628460227916946
 6600:   410.4  3.9e-02  24.28741624334893
 7700:   678.3  5.4e-02  24.224402129537033
 8800:  1876.3  7.5e-02  24.159998846857885
 9900:  6186.2  1.1e+00  22.38949467691919
evals: ax-ratio max(std)   f-value
10010:  8855.4  2.1e+00  22.216906525915686
termination by {'maxfevals': 10000}
best f-value = 22.162089522257965
solution = [-0.09577189794242238, 0.04626503745641274, 0.010784958569155928, 2.7828493750670154, -10.01307535177148, 3.8171762594501146, -0.008553308587871768, -1.3284910035805082, 0.3485022998008057, -0.015495268273185024, -0.934215037924096, 0.011513529142687638, -0.5444237016512278, 32.25225305150756]
